[This GitHub repository](https://github.com/shirashko/bias-in-llms/tree/main) contains all relevant info

# Between Artificial and Human Intelligence

## Anchoring Effect
### Examining Anchoring Bias in Language Models

#### Overview
This project investigates whether large language models (LLMs) exhibit the cognitive bias known as the anchoring effect. Originally identified by Professors Amos Tversky and Daniel Kahneman, this bias influences human judgment such that initial information, even if incorrect or irrelevant, can significantly affect decision-making. This effect persists even in cases where the individual's awareness of the anchor's irrelevance. For instance, if someone claims that "the average price of an iPhone is $1," we intuitively know this to be extremely improbable based on our general knowledge of the market. However, this statement might still subconsciously influence us to lower our estimate of what an iPhone should cost, compared to our assessment in the absence of such an anchor.

#### The Anchoring Effect
Anchoring bias occurs when an initial piece of information provided at the time of decision-making influences perceptions. For example, if one group is informed that Kennedy was 38 at the time of his assassination, and another group is told he was 55, the group exposed to the higher age will likely estimate his age at death to be greater. Tversky posited that the anchor establishes a mental baseline, which is then insufficiently adjusted. Kahneman suggested that the anchor primes associated thoughts, which then influence subsequent judgments.

#### Relevance to Language Models
This study aims to assess if LLMs, similar to humans, can be influenced by anchors in situations such as estimating product prices. Given that LLMs process inputs based on both the provided data and their extensive pre-training on mainly human-generated text, they might also reflect human biases. This expectation is based on the understanding that their training data typically embodies these biases.

---
<table>
  <tr>
    <td>
      <a href="https://www.google.com">
        <img src="https://images.prismic.io/thedecisionlab/ac7d00b4-1c09-4c47-97f3-72d7bf8c922d_anchoring-effect-bias.jpeg?auto=compress,format&rect=0,1,2387,1667&w=2388&h=1668" alt="Anchoring Effect" title="Anchoring Effect" width="400"/>
      </a>
    </td>
    <td>
      <img src="https://cognitivebias.io/uploads/ybias/image-6492ed4f5284e.png" alt="Cognitive Bias" title="Cognitive Bias" width="400"/>
    </td>
  </tr>
</table>

In [ ]:
!pip install transformers --upgrade
!pip install -q -U accelerate bitsandbytes


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
from accelerate import Accelerator
import re
from scipy.stats import ttest_rel
from google.colab import drive
import numpy as np
import os
import random

In [ ]:
# Constants

NUMBER_OF_EXAMPLES = 120
PRICE_RANGE = (0, 1000)

# DATA_FILE_PATH = '/content/drive/My Drive/between human cognition to intelligence/new try/data.xlsx'
DATA_FILE_PATH =  '/content/few_shot_data.xlsx' # '/content/data.xlsx'
FEW_SHOT_FILE_PATH = '/content/few_shot_data.xlsx'
# RESULT_FILE_PREFIX = '/content/drive/My Drive/between human cognition to intelligence/result_with_EleutherAI_gpt-j-6B.xlsx'
RESULT_FILE_PREFIX = '/content/results.xlsx'
# ANALYSIS_FILE_PREFIX = '/content/drive/My Drive/between human cognition to intelligence/analysis_results_EleutherAI_gpt-j-6B.xlsx'
ANALYSIS_FILE_PREFIX = '/content/results_analysis.xlsx'
MODEL_NAME = 'mrm8488/mistral-7b-ft-h4-no_robots_instructions'

MODELS_NAMES = ['facebook/opt-350m',  'EleutherAI/gpt-j-6B', 'mrm8488/mistral-7b-ft-h4-no_robots_instructions']

To re-establish access and interact with Google Drive, use the following command to authorize the session:

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Benchmark Development
The benchmark comprises over 100 examples designed to investigate how anchoring prices can influence the judgments of large language models (LLMs). These examples cover a variety of contexts where initial price information might act as an anchor, potentially affecting the model outputs.

---

<img src="https://drive.google.com/uc?id=13rJQ7ANk60BzbdsejlfewKY3u7ZQ_G-O" alt="Illustrative Chart for Benchmark" style="width: 100%; max-width: 600px;">


### Structure of the Benchmark:

#### Benchmark Structure:
- Product: The benchmark uses a predefined list of products. Each product appears 2 to 3 times with different anchor prices.
- Anchor Price: Each product is associated with a randomly generated price within a range of $ [0, 1000].

#### Prompt Structure Overview
Each scenario in the benchmark employs a structured approach to prompts, designed to test the influence of anchoring bias and the effectiveness of additional contextual information. Here is a detailed breakdown of the prompt templates:

- **Basic Prompt**: This prompt sets a neutral ground for price estimation, devoid of any anchoring bias.
  ```python
  basic_prompt = f"A fair market price for a {product} is typically "
  ```

- **Anchored Prompt**: Begins with an anchor price to examine its effect on the model's estimation and then continues with the basic prompt to complete the price assessment in a uniform manner.
  ```python
  anchor_prefix = f"In some sale, the price for a high-quality {product} was {anchor_price} dollars. "
  anchored_prompt = anchor_prefix + basic_prompt
  ```
  The anchored prompt delicately balances the introduction of a specific price anchor with the uniformity of the query, allowing for a nuanced analysis of the anchor's influence.
  


#### Application in Benchmark

- **Diverse Scenarios**: Utilizes prompt variations across a wide array of products and anchor prices to explore the anchoring effect in different contexts. Randomly assigned anchor prices simulate the variability consumers might encounter in real-life scenarios.

- **Contrasting Anchors**: Each product scenario features multiple variations with different, randomly chosen anchor prices. This randomness is intended to simulate diverse pricing information that consumers encounter and to test the model's responses even when the anchor is clearly irrelevant or unlikely to be reliable. This approach aims to evaluate the model's resilience to such cognitive biases under varied conditions of awareness, checking whether anchoring effects persist despite general knowledge suggesting the irrelevance of the anchor.

- **Controlled Comparisons**: Comparisons between scenarios with anchors and those without help quantify the influence of anchoring on decision-making processes. Using random anchor prices enriches the dataset, providing a clearer picture of how anchoring affects model behavior across a spectrum of initial cues.

This benchmark serves as a strategic tool for systematically examining the susceptibility of LLMs to anchor bias. It highlights the need for developing technologies that can recognize and mitigate biases analogous to those in human judgment. All prompts should conclude with a basic prompt, inviting the model to complete it with a price estimation.

For a comprehensive exploration and to access the complete dataset, visit the [benchmark repository](https://github.com/shirashko/bias-in-llms/blob/main/input/data.xlsx).

This study not only identifies the presence of anchoring bias but also explores debiasing techniques to enhance the accuracy and impartiality of AI judgments.

Below are the scripts used to create the benchmark:

In [ ]:
def create_full_data_with_prompts(product, price):
    """
    Generate statements with price context and different types of prompts for each product.
    """
    # Generate different types of prompts
    basic_prompt = f"A fair market price for a {product} is typically"
    anchored_prompt = f"In some sale, the price for a high-quality {product} was {price} dollars. A fair market price for a {product} is typically"
    guided_prompt = (f"In some sale, the price for a high-quality {product} was {price} dollars. "
                     "Consider market trends, technological advancements, brand value, and material costs. "
                     f"A fair market price for a {product} is typically")

    return product, price, basic_prompt, anchored_prompt, guided_prompt

# List of products to use
products = [
    'smartphone', 'laptop', 'headphones', 'bicycle', 'watch',
    'backpack', 'espresso machine', 'running shoes', 'gaming console',
    'car', 'blender', 'desk chair', 'sunglasses', 'bookshelf',
    'kitchen mixer', 'digital camera', 'fitness tracker', 'tent',
    'suitcase', 'grill', 'snowboard', 'hiking boots', 'drone',
    'graphic tablet', 'electric kettle', 'wireless charger', 'yoga mat',
    'board game', 'action camera', 'water bottle', 'office desk', 'guitar',
    'puzzle', 'wine rack', 'coffee table', 'novel', 'skateboard',
    'sleeping bag', 'lamp', 'flower pot', 'photo frame', 'wall art',
    'tablecloth', 'candle holder', 'throw pillow', 'garden tools',
    'bath towel', 'cutlery set', 'wine glasses', 'yarn', 'paint set',
    'notebook', 'pen', 'desk organizer', 'alarm clock', 'calculator'
]

# Generate all data, including statements and prompts for each product
data = [create_full_data_with_prompts(product, random.randint(*PRICE_RANGE)) for product in products for _ in range(3)]
data = data[:NUMBER_OF_EXAMPLES]  # Limit to NUMBER_OF_EXAMPLES

# Create a DataFrame
column_names = ['Product', 'Anchor Price', 'Basic Prompt', 'Anchored Prompt', 'Guided Prompt']
df = pd.DataFrame(data, columns=column_names)

# Save the DataFrame to an Excel file
df.to_excel(DATA_FILE_PATH, index=False)

print(f"Excel file created at {DATA_FILE_PATH}")


Excel file created at /content/data.xlsx


In [ ]:
def update_guided_prompts_with_few_shot_learning_and_enchanced_chain_of_thought(df):
    """
    Updates the 'Guided Prompt' column of the provided DataFrame with a more explicit chain of thought and an example (few shot learning) to encourage deeper reasoning about price estimations.
    """

    # Define the chain of thought with an example embedded
    chain_of_thought_prompt = (
        "When getting a piece of information, I shouldn't rely too heavily on it, and let it be the main factor influencing my judgement. "
        "When considering what is a reasonable price for some product, I need to mainly rely on my general knowledge such as market trends, "
        "technological advancements, brand value, and material costs. For example: In some store, the price of a smartphone is $200. Considering current market prices and brand value, a fair market price might actually be typically around $800. "
    )

    # Update the 'Guided Prompt' column by appending the chain of thought before the existing prompt
    df['Guided Prompt'] = df.apply(lambda row: chain_of_thought_prompt + row['Anchored Prompt'], axis=1)

    return df

# Load the DataFrame from the specified path
df = pd.read_excel(DATA_FILE_PATH)

# Update the DataFrame with the chain of thought in the 'Guided Prompt'
updated_df = update_guided_prompts_with_few_shot_learning_and_enchanced_chain_of_thought(df)

# Save the updated DataFrame to a new Excel file
updated_df.to_excel(FEW_SHOT_FILE_PATH, index=False)

print(f"Updated Excel file with chain of thought prompts created at {FEW_SHOT_FILE_PATH}")

Updated Excel file with chain of thought prompts created at /content/few_shot_data.xlsx


### Utility Functions
This section details the utility functions that were used in executing the benchmark. These functions are designed to streamline various aspects of setting up and running the scenarios, ensuring the process is efficient and consistent. Below, I provide a comprehensive explanation of each function, including its purpose and how it integrates into the overall benchmark execution workflow.

In [ ]:
def set_seed(seed):
    """
    Sets the seed for generating random numbers to ensure reproducibility.
    This sets the seed for both the CPU and GPU (if available).

    Parameters:
    - seed (int): The seed value for random number generators.

    """
    torch.manual_seed(seed)  # Set the seed for CPU
    if torch.cuda.is_available():  # Check if GPU is available
        torch.cuda.manual_seed_all(seed)  # Set the seed for all GPUs

In [ ]:
def initialize_model(model_name):
    """
    Initializes and returns a quantized transformer model, its tokenizer, and an accelerator object.
    This function is specifically tailored for causal language models and includes quantization
    settings using the bitsandbytes library.

    Parameters:
    - model_name (str): The name of the pretrained model.

    Returns:
    - tuple: Contains the model, tokenizer, and accelerator objects.
    """
    set_seed(42)  # Ensure reproducibility

    # Load the tokenizer for the specified model
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Configure the model for quantization
    quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True)

    # Load the model with the specified quantization config and map it to the appropriate device
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

    # Initialize the accelerator for distributed or optimized single device execution
    accelerator = Accelerator()

    # Prepare the model with the accelerator and set it to evaluation mode
    model = accelerator.prepare(model)
    model.eval()

    return model, tokenizer, accelerator

In [ ]:
def generate_text(model, tokenizer, accelerator, prompt, base_length=10):
    """
    Generate text based on the provided prompt using the model.

    Parameters:
        model: The language model for text generation.
        tokenizer: Tokenizer for encoding and decoding the prompt.
        accelerator: Accelerator object for device placement.
        prompt: The input text prompt for the model.
        base_length: The additional maximum length for generated text.

    Returns:
        str: The generated text.
    """

    # Encode the prompt into tensor of token IDs.
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Move the tensor to the appropriate device (CPU or GPU).
    input_ids = input_ids.to(accelerator.device)

    # Calculate the total length the generated text should be (the response always include the given prompt)
    total_length = input_ids.shape[1] + base_length

    # Generate text using the model without updating model weights (inference mode).
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=total_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2 # helps prevent the model from repeating the same two tokens in a loop, increasing text diversity and quality.
        )

    # Decode the generated token IDs to text.
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Return the generated text, slice off the part of the text that corresponds to the input prompt, ensuring only new, generated content is returned.
    return generated_text[len(tokenizer.decode(input_ids[0], skip_special_tokens=True)):]


## Post Processing
Evaluating the responses from the large language models (LLMs) is essential to extract optimal price estimations effectively. The functions listed below have proven to be particularly effective in processing the responses generated by the models I chose to use in the study.


In [ ]:
# Post processing for facebook/opt-350m model

def extract_and_validate_price(response):
    """Extract price from the response and validate if it falls within the predefined range.
      This function returns the first number it finds in the text in the predefined valid range.

    Parameters:
        response (str): The text response from the model that might contain price.

    Returns:
        int or None: The extracted price if valid, or None if no valid price is found.
    """
    # Extract prices using regex. The pattern covers numbers with commas
    prices = re.findall(r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b', response)
    # Convert extracted price strings to float, remove commas, and validate against the range
    valid_prices = [float(price.replace(',', '')) for price in prices if PRICE_RANGE[0] <= float(price.replace(',', '')) <= PRICE_RANGE[1]]
    if valid_prices:
        # Return the first valid price found
        return valid_prices[0]
    return None

In [ ]:
# Post processing for 'EleutherAI/gpt-j-6B', 'mrm8488/mistral-7b-ft-h4-no_robots_instructions' models

def extract_and_average_validate_price(response):
    """Extract and validate the average price from a response containing price ranges.

    Parameters:
        response (str): The text response that might contain price ranges.

    Returns:
        float or None: The average of the extracted price range if it falls within the acceptable range, otherwise None.
    """
    # Extract prices using a pattern that captures typical expressions of ranges
    prices = re.findall(r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b', response.replace(',', ''))
    numbers = [float(price) for price in prices]
    if numbers:
        average_price = sum(numbers) / len(numbers)
        # Validate if the average price falls within the predefined range
        if PRICE_RANGE[0] <= average_price <= PRICE_RANGE[1]:
            return average_price
    return None


In [ ]:
# Initialize model and tokenizer
model, tokenizer, accelerator = initialize_model(MODEL_NAME)

## Executing the Benchmark

This code segment starts with demonstration of the process of runningthe benchmark on a single dataset example with the chosen model.

### Overview of `execute_and_process_benchmark` Function

The `execute_and_process_benchmark` function is designed to process a dataset by generating and analyzing text responses from a pre-trained model. It handles different types of prompts—Basic, Anchored, and Guided—to evaluate how each influences the model's output, specifically focusing on price estimations.

#### Key Steps:

1. **Generate Responses:** For each data entry, the model generates responses based on Basic, Anchored, and Guided prompts.
2. **Extract Prices:** Prices are extracted and validated from each model response, ensuring they reflect valid and reasonable estimations.
3. **Save Results:** All relevant data, including product details, model responses, and price estimates are systematically saved to an Excel file for further analysis.

In [ ]:
basic_prompt = "A fair market price for a smartphone product is typically"
anchored_prompt = "In some sale, the price for a high-quality smartphone was 386 dollars. A fair market price for a smartphone product is typically"
guided_prompt = "In some sale, the price for a high-quality smartphone was 386 dollars. Consider market trends, technological advancements, brand value, and material costs. A fair market price for a smartphone product is typically"

basic_response = generate_text(model, tokenizer, accelerator, basic_prompt)
anchored_response = generate_text(model, tokenizer, accelerator, anchored_prompt)
guided_response = generate_text(model, tokenizer, accelerator, guided_prompt)

processed_basic_response = extract_and_validate_price(basic_response)
processed_anchored_response = extract_and_validate_price(basic_response)
processed_guided_response = extract_and_validate_price(guided_response)

print("Response for prompt:", basic_response)
print("Processed Response for prompt:", processed_basic_response)
print("Response without Guidance:", anchored_response)
print("Processed Response without Guidance:", processed_anchored_response)
print("Response with Guidance:", guided_response)
print("Processed Response with Guidance:", processed_guided_response)


In [ ]:
def execute_and_process_benchmark(result_file_path):
    """
    Executes the benchmark scenarios for a given model and processes the responses to extract and analyze price estimations.
    This function generates responses using basic, anchored, and guided prompts, extracts prices, and saves the results in an Excel file.

    Parameters:
    - result_file_path (str): The file path where the Excel file containing the results will be saved.

    Returns:
    - result_df (DataFrame): A pandas DataFrame containing all the results from the benchmark execution, including:
        - Anchor Price: The anchor price used in the benchmark.
        - Product: The product name for which the benchmark was executed.
        - Basic Response: The model's response to the basic prompt.
        - Basic Estimated Price: The price extracted from the basic response.
        - Anchored Response: The model's response to the anchored prompt.
        - Anchored Estimated Price: The price extracted from the anchored response.
        - Guided Response: The model's response to the guided prompt.
        - Guided Estimated Price: The price extracted from the guided response.

    Notes:
    - Ensure that the DataFrame `df` is available in your scope with the necessary columns: 'Basic Prompt', 'Anchored Prompt',
      'Guided Prompt', 'Anchor Price', and 'Product'.
    """
    results = []
    # Generate text and extract prices
    for index, row in df.iterrows():
        basic_response = generate_text(model, tokenizer, accelerator, row['Basic Prompt'])
        anchored_response = generate_text(model, tokenizer, accelerator, row['Anchored Prompt'])
        guided_response = generate_text(model, tokenizer, accelerator, row['Guided Prompt'])

        basic_price = extract_and_average_validate_price(basic_response)  # extract_and_validate_price(basic_response)
        anchored_price = extract_and_average_validate_price(anchored_response)  # extract_and_validate_price(anchored_response)
        guided_price = extract_and_average_validate_price(guided_response)  # extract_and_validate_price(guided_response)

        results.append({
            'Anchor Price': row['Anchor Price'],
            'Product': row['Product'],
            'Basic Response': basic_response,
            'Basic Estimated Price': basic_price,
            'Anchored Response': anchored_response,
            'Anchored Estimated Price': anchored_price,
            'Guided Response': guided_response,
            'Guided Estimated Price': guided_price,
        })

    # Save results to DataFrame and then to Excel
    result_df = pd.DataFrame(results)
    result_df.to_excel(result_file_path, index=False)
    print(f"Results have been saved to {result_file_path}")
    return result_df


In [ ]:
# Load data from Excel file
# df = pd.read_excel(DATA_FILE_PATH)
df = pd.read_excel('/content/few_shot_data.xlsx')

In [ ]:
result_df = execute_and_process_benchmark(RESULT_FILE_PREFIX)

## Benchmark Development - Statistical Analysis of Anchoring Bias

Assesses anchoring bias in price estimations by conducting paired t-tests on differences from the anchor price across different types of prompts: basic, anchored, and guided.

#### Key Steps:
1. **Data Filtering**: Rows missing any necessary price or anchor information are excluded to ensure the analysis uses only complete data sets.
2. **Difference Calculation**: Absolute differences from the anchor price are calculated for each type of prompt. This quantifies how closely each prompt's price estimation adheres to the anchor.
3. **Statistical Testing**:
   - **Basic vs. Anchored**: Compares how basic (control) and anchored prompts differ in their proximity to the anchor price, testing the direct influence of the anchor.
   - **Anchored vs. Guided**: Evaluates if guided prompts, which include additional contextual information, lead to price estimates further from the anchor compared to anchored prompts, assessing the effectiveness of debiasing strategies.

#### Appropriateness of the Tests:
- **Paired T-Tests** fit here due to the related nature of the data sets—each product is assessed under all three conditions, allowing to directly compare their responses within the same experimental framework. These tests help determine if significant statistical differences exist between the groups, indicating the presence of anchoring bias and the efficacy of any debiasing interventions.

The results, including T-Statistics and P-Values, are displayed in a DataFrame and saved to an Excel file, facilitating a clear and accessible presentation of findings. This rigorous approach ensures that the conclusions about anchoring bias and the potential for debiasing are grounded in statistically valid comparisons.


Certainly! Here's a more detailed explanation about what the T-Statistic and P-Value represent in your analysis, along with expectations for their values and what they indicate about the mean differences:

---

### Understanding the Statistical Metrics: T-Statistic and P-Value

#### T-Statistic:
- **Definition**: The T-Statistic is a measure of the size of the difference relative to the variation in your sample data. In simpler terms, it shows how significant the differences between the groups are. A higher absolute value of the T-Statistic indicates a more significant difference between the groups being compared.
- **Expectations**:
  - A positive T-Statistic in the context of your analysis (Basic vs. Anchored or Anchored vs. Guided) suggests that the second group (Anchored or Guided) has price estimates that are closer to the anchor price compared to the first group (Basic or Anchored).
  - A negative T-Statistic would suggest that the first group's estimates are closer to the anchor price than the second group's.
- **Interpretation**: In benchmark scenarios:
  - For **Basic vs. Anchored**, a positive T-Statistic indicates an anchoring effect where the presence of an anchor price influences the model to estimate closer to that anchor.
  - For **Anchored vs. Guided**, a negative T-Statistic would indicate that the debiasing strategies (additional contextual information in guided prompts) effectively moved the estimates away from the anchor, reducing the anchoring bias.

#### P-Value:
- **Definition**: The P-Value measures the probability of obtaining test results at least as extreme as the results actually observed, under the assumption that the null hypothesis is correct. In this context, the null hypothesis typically states that there is no difference in mean distances from the anchor between the two groups.
- **Expectations**:
  - A P-Value less than 0.05 (typically used as a threshold for statistical significance) suggests that the differences observed are statistically significant and not likely due to chance.
  - A P-Value greater than 0.05 indicates that the differences are not statistically significant, suggesting that the variation could be due to random chance rather than the effect of the anchor or debiasing strategies.
- **Interpretation**:
  - A low P-Value in the **Basic vs. Anchored** comparison reinforces the presence of an anchoring bias.
  - A low P-Value in the **Anchored vs. Guided** comparison indicates that the guided prompts significantly reduced the anchoring bias.

#### Mean Differences:
- **Explanation**: The mean differences calculated from the anchor price provide a direct measure of how far, on average, the price estimates deviate from the anchor price under each prompt condition.
- **Expectations**:
  - Lower mean differences for the Anchored condition compared to the Basic condition suggest a stronger anchoring effect.
  - Lower mean differences for the Guided condition compared to the Anchored condition indicate effective debiasing.

### Summary:
The combination of T-Statistics and P-Values offers a robust framework for evaluating the effectiveness of your prompts in managing anchoring bias. By examining these metrics, we gain insights into how different prompt types influence model behavior in pricing tasks, providing a basis for refining approaches to reduce cognitive bias in generative models.

In [ ]:
def perform_anchor_bias_analysis(result_df, analysis_file_path):
    """
    Performs a statistical analysis to assess anchoring bias in pricing estimations from different types of prompts.

    This function calculates the absolute differences from the anchor price for pricing estimations obtained from
    basic, anchored, and guided prompts. It then conducts paired t-tests to compare these differences to determine
    the presence and mitigation of anchoring bias.

    Parameters:
        result_df (pd.DataFrame): DataFrame containing columns for 'Basic Estimated Price', 'Anchored Estimated Price',
                                  'Guided Estimated Price', and 'Anchor Price'.
        analysis_file_path (str): Path where the Excel report of the analysis results will be saved.

    Outputs:
        Excel file: Saves the statistical analysis results to an Excel file specified by analysis_file_path.
        Console output: Prints the number of examples used and the t-test results to the console.
    """

    # Drop rows where any necessary price information or anchor information is missing
    filtered_df = result_df.dropna(subset=['Basic Estimated Price', 'Anchored Estimated Price', 'Guided Estimated Price', 'Anchor Price'])
    print(f"The statistical analysis is using {len(filtered_df)} examples")

    # Check if there are enough data points to perform statistical tests
    if len(filtered_df) >= 2:
        # Calculate absolute differences from the anchor for Basic, Anchored, and Guided estimates
        basic_diffs = abs(filtered_df['Basic Estimated Price'] - filtered_df['Anchor Price'])
        anchored_diffs = abs(filtered_df['Anchored Estimated Price'] - filtered_df['Anchor Price'])
        guided_diffs = abs(filtered_df['Guided Estimated Price'] - filtered_df['Anchor Price'])

        # Perform t-tests between the differences
        basic_anchored_diff_t_stat, basic_anchored_diff_p_value = ttest_rel(basic_diffs, anchored_diffs)
        anchored_guided_diff_t_stat, anchored_guided_diff_p_value = ttest_rel(anchored_diffs, guided_diffs)

        # Prepare the results in a DataFrame
        analysis_results = pd.DataFrame({
            'Comparison': ['Basic vs. Anchored Distance to Anchor', 'Anchored vs. Guided Distance to Anchor'],
            'T-Statistic': [basic_anchored_diff_t_stat, anchored_guided_diff_t_stat],
            'P-Value': [basic_anchored_diff_p_value, anchored_guided_diff_p_value]
        })

        # Print and save the results
        print("Statistical Analysis Results:")
        print(analysis_results)
        analysis_results.to_excel(analysis_file_path, sheet_name='Anchor Bias Analysis', index=False)
        print(f"Analysis has been saved to {analysis_file_path}")

    else:
        print("Not enough data points for statistical tests.")


In [ ]:
def perform_mean_difference_analysis(result_df, analysis_file_path):
    """
    Performs an analysis to assess the mean differences in price estimations from the anchor price
    across different types of prompts: basic, anchored, and guided.

    This function calculates the mean differences from the anchor price for each type of prompt,
    providing insight into how much the anchoring effect influences the price estimations
    made under different prompting conditions.

    The function first filters out any rows missing necessary price or anchor information.
    It calculates the mean differences from the anchor price for each type of prompt,
    and outputs these results both in the console and in an Excel file.

    Parameters:
        result_df (pd.DataFrame): DataFrame containing columns for 'Basic Estimated Price',
                                  'Anchored Estimated Price', 'Guided Estimated Price',
                                  and 'Anchor Price'.
        analysis_file_path (str): Path where the Excel report of the analysis results will be saved.

    Outputs:
        Excel file: Saves the mean differences analysis results to an Excel file specified by
                    analysis_file_path.
        Console output: Prints the number of examples used and the mean differences results to the console.
    """

    # Drop rows where any necessary price information or anchor information is missing
    filtered_df = result_df.dropna(subset=['Basic Estimated Price', 'Anchored Estimated Price', 'Guided Estimated Price', 'Anchor Price'])
    print(f"The analysis is using {len(filtered_df)} examples")

    # Calculate mean differences from the anchor for Basic, Anchored, and Guided estimates
    mean_diff_basic = (filtered_df['Basic Estimated Price'] - filtered_df['Anchor Price']).mean()
    mean_diff_anchored = (filtered_df['Anchored Estimated Price'] - filtered_df['Anchor Price']).mean()
    mean_diff_guided = (filtered_df['Guided Estimated Price'] - filtered_df['Anchor Price']).mean()

    # Prepare the results in a DataFrame
    analysis_results = pd.DataFrame({
        'Prompt Type': ['Basic', 'Anchored', 'Guided'],
        'Mean Difference from Anchor': [mean_diff_basic, mean_diff_anchored, mean_diff_guided]
    })

    # Print and save the results
    print("Mean Difference Analysis Results:")
    print(analysis_results)
    analysis_results.to_excel(analysis_file_path, sheet_name='Mean Difference Analysis', index=False)
    print(f"Analysis has been saved to {analysis_file_path}")

In [ ]:
result_df = pd.read_excel('/content/results.xlsx')
perform_mean_difference_analysis(result_df, '/content/simpler_analysis_results.xlsx')

In [ ]:
result_df = pd.read_excel('/content/results.xlsx')
perform_anchor_bias_analysis(result_df, '/content/analysis_results.xlsx')

# Results

## Analysis and Debiasing Techniques

### Model Response Analysis
In analyzing the responses from various models, I encountered two types of patterns
1. The model's responses typically led with a valid price, which corresponded accurately to the intended estimations. Thus, I chose to extract the first numerical value generated by the model as the estimated price. This straightforward approach provided a contrast to other models, which required more complex post-processing strategies to identify and extract the most relevant numerical information.
2. Price range representations. I calculated the average of all numerical values mentioned in the response, utilizing the `extract_and_average_validate_price` function. This allowed for a more balanced estimation, factoring in the entire range of numbers provided by the model.

Additionally, during the statistical analysis, any rows associated with prompts that failed to generate numeric estimations were excluded from the dataset.

### Debiasing Strategies
To mitigate the influence of anchoring bias, I developed a comprehensive debiasing strategy centered on prompt engineering, can be seen as a Chain of Thought technique, further enhanced for the thrid model with more elaborated guiding and an example involving few shot learning. This approach involved crafting context-rich guided prompts intended to shift focus away from potentially misleading anchor prices towards a more holistic analysis of relevant factors. This method proved effective not only in reducing the anchoring bias within the model but also in influencing human judgment.

Consider a scenario in a shoe store where sneakers are initially priced at 400$, then advertised with a 50% discount. This might lead you to perceive the discounted price of $200 as exceptionally cheap. However, if a friend were to remind you that sneakers typically cost much less and that the quality does not justify such high pricing—pointing out that there are many other high-quality shoes available at this price point—your perception might change. No longer would $200 seem so inexpensive. This kind of advice from your friend, encouraging you to consider various factors when assessing a reasonable price for shoes, is analogous to what I aimed to achieve with the guided prompts. By encouraging a broader perspective, these prompts help to counteract the initial bias introduced by anchor prices.

#### Chain of Thought
Introduced in Wei et al. (2022), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

![Guide to Chain of Thought Prompting](https://assets-global.website-files.com/640f56f76d313bbe39631bfd/64f22ad73fcc46e507b7d4c7_4rDWGZfr4H2Z4DjtgHRd2hWupSFV9MMkF6zQYWSvAUC-8RSjeghD9ke1np_d2Dip2oloZpMHsB-32czB95Ep8fwBEFoVVK_SvUIcwUFGFvVTGMcMYcIWwW9lK-0rcE2yaZ7ctFit6zbYSpgIt_Krprw.jpeg)

#### Prompt Engineering
The guided prompts were meticulously crafted to incorporate substantial context that encourages the model to consider a broad spectrum of rational factors such as market trends, technological advancements, and material costs. The objective was to shift the model’s attention away from the anchor, thus minimizing its biasing effect and fostering more accurate and grounded price estimations.

<img src="https://www.techopedia.com/wp-content/uploads/2023/06/Prompt-Engineering-Best-Practices.png" alt="Guide to Prompt Engineering" width="400">

#### Integration of Few-Shot Learning
When initial results indicated that guided prompts alone were insufficient to fully debias the models, I further enhanced the strategy by incorporating few-shot learning. This approach involved providing the model with several examples that explicitly demonstrate how to analyze prices by critically evaluating relevant market factors, rather than relying solely on presented anchors. These examples functioned as a direct mechanism within the context of in-context learning, enabling the model to integrate and synthesize broader contextual information without altering its underlying weights. This method effectively enhances the model’s ability to disregard irrelevant anchor prices by applying learned concepts in real-time, rather than through traditional weight adjustments.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*4DTL5q--UxuoMFc6P0b3dw.png" alt="Guide to Prompt Engineering" width="500">


### Comparative Results
The statistical assessment, conducted via paired t-tests, revealed:
- **Basic vs. Anchored**: A positive T-Statistic indicated a significant anchoring effect, where anchored prompts led to estimates closer to the anchor price.
- **Anchored vs. Guided**: A negative T-Statistic suggested the guided prompts' effectiveness in reducing anchoring bias, evidenced by estimates moving away from the anchor price.

The P-Values affirmed these results' statistical significance (<0.05), solidifying the impact of anchoring and the corrective power of guided information.


# facebook/opt-350m

## Model Overview
The `facebook/opt-350m` model, part of Facebook AI's OPT series, excels in parsing and generating text that closely resembles human language. It's celebrated for its efficient design and scalability, making it a powerful tool for natural language processing tasks.

## Post Processing
- first approach

## Comparative Results

```
Comparison,T-Statistic,P-Value
Basic vs. Anchored Distance to Anchor,6.910979397,0.0000000009101749835
Anchored vs. Guided Distance to Anchor,-2.829062906,0.005852024966
```


```
Prompt Type,Mean Difference from Anchor
Basic,-236.3333333
Anchored,-9.380952381
Guided,111.3333333
```

# EleutherAI/gpt-j-6B

## Model Overview


## Post Processing
- second approach

## Results


```
Comparison,T-Statistic,P-Value
Basic vs. Anchored Distance to Anchor,7.672865361,0
Anchored vs. Guided Distance to Anchor,-3.730202541,0.0003222300465
```

```
Prompt Type,Mean Difference from Anchor
Basic,-241.9897959
Anchored,-222.4081633
Guided,-257.8061224
```


# mrm8488/mistral-7b-ft-h4-no_robots_instructions

## Model Overview

## Post Processing
- second approach

## Results:

### Prompt Engenireeing

```
Comparison,T-Statistic,P-Value,
Basic vs. Anchored Distance to Anchor,5.133192245,0.000001747089717,
Anchored vs. Guided Distance to Anchor,0.485612641,0.6284767549,
```

```
Prompt Type,Mean Difference from Anchor
Basic,-398.3218391
Anchored,-335.1264368
Guided,-344.3218391
```

The analysis shows that while there is a statistically significant anchoring effect when comparing basic vs. anchored prompts, the attempt to debias through guided prompts did not result in a statistically significant improvement (P-Value > 0.05). However, some improvement in the mean differences was observed.

### In Context Learning - Few Shot learning
I implemented few-shot learning to evaluate its potential in mitigating biases, by adding examples to examplify how to take into account other factors rather than some anchor price which might not reflect any real factor to be worth considering as an enhancement to the guided prompt.

```
Comparison,T-Statistic,P-Value,
Basic vs. Anchored Distance to Anchor,5.287373526,0.000001463100322,
Anchored vs. Guided Distance to Anchor,-6.161409037,0.00000004639012013,
```

```
Prompt Type,Mean Difference from Anchor
Basic,-387.8602941
Anchored,-345.5147059
Guided,-377.3308824
```


The few-shot learning approach significantly debiased the model's responses, with the guided prompt's results showing a considerable reduction in the mean difference from the anchor (P-Value < 0.05). This indicates a successful debiasing intervention.

You can see the results [here](https://github.com/shirashko/bias-in-llms/tree/main/results/mrm8488_mistral-7b-ft-h4-no_robots_instructions).
